# Descargar Imagenes

Esta función carga imágenes desde subcarpetas específicas (glioma, meningioma, pituitary, notumor) dentro de un directorio dado. Redimensiona cada imagen al tamaño especificado y asigna una etiqueta numérica (0, 1, 2 o 3) a cada imagen según la clase a la que pertenece (nombre de la carpeta).

Argumentos:

* path (str): La ruta del directorio principal que contiene las subcarpetas de imágenes.
* tamaño (tuple): Una tupla (ancho, alto) que especifica las dimensiones a las que se redimensionarán las imágenes. Por defecto es (100, 100).

Retorna:
* imagenes (list): Una lista de arrays de imágenes redimensionadas y convertidas a RGB.
* etiquetas (list): Una lista de listas, donde cada sublista contiene la etiqueta numérica correspondiente a cada imagen.
* nombres (list): Una lista de los nombres de archivo originales de cada imagen.

In [ ]:
def cargar_imagenes(path, tamaño=(100, 100)):
    """
    Carga imágenes desde las carpetas (glioma, meningioma, pituitary, notumor),
    las redimensiona y asigna etiquetas del 0 al 3 según la clase.
    """
    imagenes = []
    etiquetas = []
    nombres = []

    # Mapear nombres de carpeta a etiquetas numéricas
    clases = {'glioma': 0, 'meningioma': 1, 'pituitary': 2, 'notumor': 3}

    carpetas = [os.path.join(path, carpeta) for carpeta in os.listdir(path)
                if os.path.isdir(os.path.join(path, carpeta))]

    for carpeta in carpetas:
        nombre_clase = os.path.basename(carpeta).lower()
        etiqueta = clases.get(nombre_clase)
        if etiqueta is None:
            continue  # Ignora carpetas no reconocidas

        archivos = os.listdir(carpeta)
        for archivo in tqdm(archivos, desc=f"Cargando {nombre_clase}"):
            ruta = os.path.join(carpeta, archivo)
            if ruta.lower().endswith(('.jpg', '.jpeg', '.png')):
                img = cv2.imread(ruta)
                if img is not None:
                    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    img_resized = cv2.resize(img_rgb, tamaño)
                    imagenes.append(img_resized)
                    etiquetas.append([etiqueta])
                    nombres.append(archivo)

    return imagenes, etiquetas, nombres

#Resolución minima

Esta función recorre las subcarpetas de un directorio dado para encontrar la imagen con la resolución más baja (menor cantidad total de píxeles).

Argumentos:

ruta (str): La ruta del directorio principal que contiene las subcarpetas de imágenes.

Retorna:

mejor_dim (tuple): Una tupla (ancho, alto) que representa las dimensiones de la imagen con la resolución más baja encontrada.

mejor_img (str): La ruta completa del archivo de la imagen con la resolución más baja.

In [ ]:
def obtener_resolucion_minima(ruta):
    min_res = float('inf')
    mejor_dim = (0, 0)
    mejor_img = ""

    for carpeta in listdir(ruta):
        ruta_carpeta = join(ruta, carpeta)
        for archivo in listdir(ruta_carpeta):
            ruta_img = join(ruta_carpeta, archivo)
            if ruta_img.lower().endswith(('.jpg', '.jpeg', '.png')):
                img = cv2.imread(ruta_img)
                if img is not None:
                    alto, ancho = img.shape[:2]
                    res_actual = alto * ancho
                    if res_actual < min_res:
                        min_res = res_actual
                        mejor_dim = (ancho, alto)
                        mejor_img = ruta_img
    return mejor_dim, mejor_img

# Ratio de las imagenes

Analiza el aspecto ratio (ancho/alto) de todas las imágenes en una carpeta.

Argumentos:
* ruta_carpeta_imagenes (str): La ruta al directorio que contiene las imágenes.

Retorna: Un tuple que contiene:
* float: El aspecto ratio promedio.
* float: La desviación estándar de los aspectos ratios.
* list: Una lista de todos los aspectos ratios calculados.
* int: El número total de imágenes procesadas.

In [ ]:
def analizar_aspecto_ratio_imagenes(ruta_carpeta_imagenes):
    aspect_ratios = []
    num_imagenes_procesadas = 0
    extensiones_validas = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.tif') # Puedes añadir más si es necesario

    print(f"Analizando imágenes en: {ruta_carpeta_imagenes}")

    for nombre_archivo in os.listdir(ruta_carpeta_imagenes):
        if nombre_archivo.lower().endswith(extensiones_validas):
            ruta_completa = os.path.join(ruta_carpeta_imagenes, nombre_archivo)
            try:
                # Cargar la imagen en modo BGR (color) o escala de grises.
                # Asegúrate de que tus imágenes tengan la misma cantidad de canales
                # Si tus imágenes son en escala de grises, puedes usar cv2.IMREAD_GRAYSCALE
                imagen = cv2.imread(ruta_completa, cv2.IMREAD_COLOR) # O cv2.IMREAD_UNCHANGED para mantener canales originales

                if imagen is None:
                    print(f"Advertencia: No se pudo cargar la imagen {nombre_archivo}. Ignorando.")
                    continue

                # Obtener las dimensiones de la imagen
                alto, ancho = imagen.shape[0], imagen.shape[1]

                if alto == 0 or ancho == 0:
                    print(f"Advertencia: Imagen con dimensiones cero {nombre_archivo}. Ignorando.")
                    continue

                aspect_ratio = ancho / alto
                aspect_ratios.append(aspect_ratio)
                num_imagenes_procesadas += 1

            except Exception as e:
                print(f"Error procesando {nombre_archivo}: {e}")
                continue

    if not aspect_ratios:
        print("No se encontraron imágenes válidas para procesar.")
        return 0, 0, [], 0

    promedio_ratio = np.mean(aspect_ratios)
    desviacion_estandar_ratio = np.std(aspect_ratios)

    return promedio_ratio, desviacion_estandar_ratio, aspect_ratios, num_imagenes_procesadas
